In [1]:
import json
import csv
import pandas as pd
from random import randint
import numpy as np
import torch
import scipy
import scipy.spatial
import fasttext
import time

In [2]:
test_set_size = 300

## Load Model

In [3]:
# Add other fastText pre-trained models url
# https://github.com/facebookresearch/fastText/blob/master/docs/pretrained-vectors.md
# https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c

# word_embedders.get_fasttext()

In [4]:
import sister

start_time = time.time()

sentence_embedding = sister.MeanEmbedding(lang="en")

print("--- %s seconds ---" % (time.time() - start_time))

Loading model...
--- 6.864881992340088 seconds ---


In [ ]:
sentence = "I am a dog."
vector = sentence_embedding(sentence)

## Path Set

In [ ]:
books_description_path = '../final_proj/cleandata/description.csv'
ground_truth_similar_books_csv = "cleandata/similar_books.csv"


## Generate Test Set

In [5]:
test_set_size = 300
df_similar_books_GT = pd.read_csv(ground_truth_similar_books_csv)

In [57]:
def get_test_set_similar_books_GT():
    from random import seed
    from random import randint
    seed(10)
    
    similar_books_GT_test_dict = {}
    count = 0
    i = 0
    while count < 300:
#         i = randint(0, len(df_similar_books_GT)) # add the randomness of the test set.
        temp_similar_books = df_similar_books_GT.iloc[i]['similar_books']
        if temp_similar_books != "[]":
            book_id = df_similar_books_GT.iloc[i]['book_id']
            # preprocess the similar books GT
            temp_similar_books = temp_similar_books.replace("[", "")
            temp_similar_books = temp_similar_books.replace("]", "")
            temp_similar_books = temp_similar_books.replace("'", "")
            temp_similar_books = [int(j.strip()) for j in temp_similar_books.split(",")]
            similar_books_GT_test_dict[book_id] = temp_similar_books
            count = count+1
        i = i+1
    return similar_books_GT_test_dict
# similar_books_GT_test_dict= get_test_set_similar_books_GT()
# similar_books_GT_test_dict[29245852]

## Generate Similarity Score

In [8]:
def generate_similarity_score(text1,text2):
    embedding1 = sentence_embedding(text1)
    embedding2 = sentence_embedding(text2)
    distance = scipy.spatial.distance.cosine(embedding1, embedding2)
    return distance

## Import Description table

In [10]:
df_description = pd.read_csv(books_description_path)
df_description = df_description.replace(np.nan, "", regex=True)

In [11]:
df_description.head()

,book_id,description
0,5333265,
1,1333909,"Anita Diamant's international bestseller ""The ..."
2,7327624,Omnibus book club edition containing the Ladie...
3,6066819,Addie Downs and Valerie Adler were eight when ...
4,287140,


In [12]:
def get_book_description(book_id):
    return df_description[df_description['book_id'] == book_id]['description'].to_string(index=False).strip()

## Import book-to-book Similar Set Table

In [ ]:
# not using it for now

## Test Performance

In [58]:
closest_n=20

similar_books_GT_test_dict = get_test_set_similar_books_GT()
accuracy_set = [] 
accuracy_closest_5_set = []

for book_id in similar_books_GT_test_dict.keys():
    distances = []
    book_description = get_book_description(book_id)
    if book_description == '':
        continue
    for similar_book_ids in similar_books_GT_test_dict[book_id]:
        similar_book_description = get_book_description(similar_book_ids)
        if book_description != '' and similar_book_description!= '':
            distance  = generate_similarity_score(book_description, similar_book_description)
            distances.append(distance)
        if len(distances)>20:
            break
            
    distances.sort()
    distances_closet_5 = distances[:min(5,len(distances))]
    
    accuracy = 1 - sum(distances)/len(distances)
    accuracy_closest_5 = 1 - sum(distances_closet_5)/len(distances_closet_5)
    
    accuracy_set.append(accuracy)
    accuracy_closest_5_set.append(accuracy_closest_5)


In [59]:
accuracy = sum(accuracy_set)/len(accuracy_set)

In [60]:
accuracy_closest_5_set = sum(accuracy_closest_5_set)/len(accuracy_closest_5_set)

In [61]:
accuracy

0.751763024560444

In [62]:
accuracy_closest_5_set

0.7866675584652593

In [ ]:
similar_books_GT_test_dict

In [ ]:
def get_test_set_similar_books_GT():
    from random import seed
    from random import randint
    seed(10)
    
    similar_books_GT_test_dict = {}
    
    for _ in range(test_set_size):
        i = randint(0, len(df_similar_books_GT)) # add the randomness of the test set.
        temp_similar_books = df_similar_books_GT.iloc[i]['similar_books']
        if temp_similar_books != "[]":
            book_id = df_similar_books_GT.iloc[i]['book_id']
            # preprocess the similar books GT
            temp_similar_books = temp_similar_books.replace("[", "")
            temp_similar_books = temp_similar_books.replace("]", "")
            temp_similar_books = temp_similar_books.replace("'", "")
            temp_similar_books = [int(j.strip()) for j in temp_similar_books.split(",")]
            similar_books_GT_test_dict[book_id] = temp_similar_books
    return similar_books_GT_test_dict

similar_books_GT_test_dict= get_test_set_similar_books_GT()
similar_books_GT_test_dict[29245852]

In [ ]:
model_flag = "fastTest"
def get_accuracy(model_flag):
    test_set_accuracy = 0
    test_set_accuracy_closest_5 = 0
    count_iteration = 0
    # get the test set description
    for book_id, similar_books_id_list in similar_books_GT_test_dict.items():
        description_book, description_similar_books_list = get_description_for_book_and_similar_books(book_id, similar_books_id_list)
        # print(description_book)
        # print(len(description_similar_books_list))
        if len(description_book) < 20: # filter out those with less description to keep the model accurate
            continue
        
        # fastTestt
        if model_flag == "fastTest":
            temp_accuracy, temp_accuracy_closest_5 = generate_embeddings_and_compute_ranked_distance(description_book, description_similar_books_list, print_mode=False)
      
            if temp_accuracy > 0: # filter out input book description is null or meaningless
                test_set_accuracy += temp_accuracy
                test_set_accuracy_closest_5 += temp_accuracy_closest_5
                count_iteration += 1

    test_set_accuracy /= count_iteration
    test_set_accuracy_closest_5 /= count_iteration
    
    return test_set_accuracy, test_set_accuracy_closest_5

test_set_accuracy, test_set_accuracy_closest_5 = get_accuracy(model_flag)
print('---------------------------------')
print('test_set_accuracy: ', test_set_accuracy)
print('test_set_accuracy_closest_5: ', test_set_accuracy_closest_5)

In [ ]:
with open('description_embedding_fasttext.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['book_id', 'embedding'])
    lines_with_error_count = 0
    
    with open(file_path) as file:
        next(file)
        for line in file:
            try:
#                 data = line.split()
#                 book_id = data[1]
#                 embedding = sentence_embedding(data[2])
#                 writer.writerow([book_id, embeddingii])
                data = line.split()
                book_id = data[0]
                if book_id == 287141:
                    a = (data[1])
                break
            except:
                lines_with_error_count += 1


In [ ]:
with open(file_path) as file:
    next(file)
    for line in file:
        try:
#                 data = line.split()
#                 book_id = data[1]
#                 embedding = sentence_embedding(data[2])
#                 writer.writerow([book_id, embeddingii])
            data = line.split('')
            book_id = data[0]
            a = data[1]
            break
        except:
            lines_with_error_count += 1



In [ ]:
a

In [ ]:
del a

## Calculate Similarity Score

In [ ]:
lines_with_error_count
scipy.spatial.distance.cosine(vector, vector)

In [ ]:
a =  generate_similarity_score(vector,vector)

Format

https://fasttext.cc/docs/en/english-vectors.html

The first line of the file contains the number of words in the vocabulary and the size of the vectors. Each line contains a word followed by its vectors, like in the default fastText text format. Each value is space separated. Words are ordered by descending frequency. These text models can easily be loaded in Python using the following code:

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [ ]:
data = load_vectors('pretrained/crawl-300d-2M.vec')

In [ ]:
import fasttext